## Exemplo Real: Regressão Logística

Lembrando do modelo: dado $X \subset R^n$, e $Y \in \{0, 1\}$, queremos modelar $P(Y | X) $

Ajustamos variáveis $w \in R^n$ e $b \in R$ em uma função $\phi(w^T x + b)$ com $\phi$ sendo a função logística: $\phi(x) = \frac{1}{1 + e^{-x}}$ de modo a aproxima-la à distribuição dos dados. 

Calculamos a distância da nossa função aos dados por meio de uma função de erro, como a entropia cruzada

A regressão logística não tem fórmula fechada como a regressão linear, e então precisa ser aproximada iterativamente com, por exemplo, gradiente descendente, algo bem próximo do que vamos fazer depois com redes neurais



In [8]:
import numpy as np
import theano
import theano.tensor as T
rng = numpy.random

#Geramos um dataset aleatório
N     = 400  # número de samples
feats = 80  # dimensão de X

reg = 0.01 
lrate = 0.1
train_steps = 10000

D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))

In [2]:
#Declaramos as variáveis do nosso modelo
w = theano.shared(rng.randn(feats), name='w')
b = theano.shared(0., name='b')

print("Modelo inicial:")
print(w.get_value())
print(b.get_value())

Modelo inicial:
[ 1.00511392 -0.57689606  1.41667788 -0.63896681  0.02442599  1.88308126
  0.26708231 -1.35224373  0.33745236 -1.08164057 -0.37494809  0.27911922
 -0.63412044 -1.93504213  0.30241739  0.24547618  0.3333101  -0.39827942
 -0.39572764  0.78540794 -0.12079154 -0.6304335  -1.5673144   0.95311307
 -0.58019301  0.27699765 -0.60386652 -0.16089546 -0.33377906  1.56189061
  0.64754723  1.3102639   2.1094359   0.43391526 -0.70837611  1.33998524
  0.32193559 -1.46285696  0.17470696 -0.84009024  0.89783436  0.94069505
  1.4287452   1.01395093 -0.70399384 -0.17354183  0.97517412  0.00905943
 -1.25402882  0.25748479 -0.9070115   1.32655534  1.4118332  -1.52355775
  0.83309389  1.29165229 -0.03453613 -0.18436934  0.21752453 -0.70230508
  0.29193562  0.34958274 -0.24258097 -0.32193636 -1.03638478 -0.65307213
 -0.4749035   0.55112005 -2.33509842 -0.27948956 -0.76870945  1.06865385
  1.36955472  0.23340198  1.13320014  0.82380872  0.67688965 -0.59377532
  0.67489563  0.16517978]
0.0


In [6]:
#Declaramos as entradas e construímos o grafo
x = T.dmatrix('x')
y = T.dvector('y')

#grafo de computação
prob = 1 / (1 + T.exp(-T.dot(x, w) - b)) #funcao logistica 
xent = -y * T.log(prob) - (1-y) * T.log(1-prob)  #entropia cruzada como funcao de erro
cost = xent.mean() + reg * (w ** 2).sum() #função de custo com regularização

#gradientes que precisamos para atualizar
gw = T.grad(cost, w) 
gb = T.grad(cost, b)

pred = prob > 0.5 #classificação final

train = theano.function(
            inputs =[x,y],
            outputs=[cost],
            updates =((w, w - lrate * gw), (b, b - lrate * gb))
)
predict = theano.function(inputs=[x], outputs=pred)

In [9]:
#Treinamos 
for i in range(train_steps):
    err = train(D[0], D[1])
    
#Verificamos
score = np.mean(predict(D[0]) == D[1])
print(score)

#Vemos nossos pesos se estivermos interessados neles
print(w.get_value())
print(b.get_value())

0.7
[ 0.2352585   0.05271406  0.0843816   0.10636819  0.09777833  0.11056372
 -0.13924167  0.03658262  0.03088404 -0.17202137 -0.00197768 -0.23786841
  0.08404153  0.01693306 -0.02586031  0.13434174  0.00503928  0.12976716
  0.00726391 -0.16474577  0.00993384 -0.17290333  0.12692331  0.18377344
 -0.02534364 -0.0309019   0.02290344 -0.01197623 -0.08396446 -0.09888248
  0.08702242  0.16960937  0.11459188  0.24938748  0.02360551 -0.0669684
 -0.01088191  0.05708302  0.01405858 -0.00949977 -0.11928919 -0.08322107
  0.21922813  0.15327323  0.13301169 -0.01100598  0.187169    0.18598729
 -0.31410631  0.00174422  0.15419301 -0.08304781  0.14571943  0.00391978
  0.07481458 -0.13329394 -0.03929186 -0.00855504  0.24578408 -0.13181403
 -0.05334212 -0.12521717 -0.07812063 -0.00387871  0.22195901 -0.07407684
  0.21572049 -0.23840053  0.12834404 -0.10668704 -0.08806761  0.28870504
 -0.08108954  0.11625554 -0.00861516 -0.03074065  0.0924149   0.02623087
 -0.12662554  0.01133747]
-0.001227776603438725
